In [1]:
import pickle

import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import sparse
from itertools import product

from scipy.sparse import hstack
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor
from sklearn.metrics import roc_curve

PATH_DATA = 'chgk/'

In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1 Знакомство с данными / предобработка

In [3]:
with open(PATH_DATA + 'players.pkl', 'rb') as f:
    players = pickle.load(f)
    
with open(PATH_DATA + 'results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open(PATH_DATA + 'tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [4]:
print(len(players.keys()), 'Число игроков')
print('Пример', players[1])
df_players = pd.DataFrame(players).T

204063 Число игроков
Пример {'id': 1, 'name': 'Алексей', 'patronymic': None, 'surname': 'Абабилов'}


In [5]:
df_players.head()

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


In [6]:
print(len(results.keys()), 'Число результатов')

5528 Число результатов


In [7]:
print(len(tournaments.keys()), 'Число турнаментов')
print('Пример', tournaments[2])

5528 Число турнаментов
Пример {'id': 2, 'name': 'Летние зори', 'dateStart': '2003-08-09T00:00:00+04:00', 'dateEnd': '2003-08-09T00:00:00+04:00', 'type': {'id': 2, 'name': 'Обычный'}, 'season': '/seasons/1', 'orgcommittee': [], 'synchData': None, 'questionQty': None}


In [8]:
df_tournaments = pd.DataFrame(tournaments).T

In [9]:
df_tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [10]:
df_results = pd.concat(
    [
        pd.concat(
            [pd.DataFrame(results[key]), pd.DataFrame({'id': [key] * len(results[key])}, dtype=int)],
            axis=1
        ) for key in results.keys()
    ]
)

In [11]:
df_results.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id
0,"{'id': 242, 'name': 'Команда Азимова', 'town':...",None,"{'name': 'Команда Азимова', 'town': {'id': 21,...",0.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",1
1,"{'id': 640, 'name': 'Перезагрузка', 'town': {'...",None,"{'name': 'Перезагрузка', 'town': {'id': 100, '...",0.0,None,2.0,[],[],[],1
2,"{'id': 245, 'name': 'Айастан', 'town': {'id': ...",None,"{'name': 'Айастан', 'town': {'id': 100, 'name'...",0.0,None,3.5,[],[],[],1
3,"{'id': 299, 'name': 'Команда Гусейнова', 'town...",None,"{'name': 'Команда Гусейнова', 'town': {'id': 2...",0.0,None,3.5,[],[],[],1
4,"{'id': 1189, 'name': 'Грааль', 'town': {'id': ...",None,"{'name': 'Грааль', 'town': {'id': 369, 'name':...",0.0,None,5.0,[],[],[],1


Выберу только те результаты игр для которых есть  данные о составе команд и повопросные результаты

In [12]:
df_results = df_results[~df_results['mask'].isna()]

In [13]:
df_results.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id
0,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
1,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
3,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
4,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


Добавлю информацию о начале турнира

В тренировочный набор данных пойдут данные за 2019 год, а за 2020 будут в тесте.

In [14]:
df_results = df_results.merge(df_tournaments, how='left')

In [15]:
for col in ['dateStart', 'dateEnd']:
    df_results[col] = pd.to_datetime(pd.to_datetime(df_results[col]), utc=True)

In [16]:
df_results = df_results[df_results['dateStart'].apply(lambda x: x.year).isin((2019, 2020))]

Буду считать, что вопросы на которые не было однозначного ответа явлются не отвеченными. Такие вопросы обозначены символом 'X' и '?'. 

In [17]:
df_results['mask'] = df_results['mask'].apply(lambda x: [int(v) for v in x.replace('X', '0').replace('?', '0')])

In [18]:
df_results['playersId'] = df_results['teamMembers'].apply(lambda x: [player['player']['id'] for player in x])

Отберу только тех игроков которые участвовали и в 2019 и в 2020 годах. Команды чей состав поредеет из игры не выбывают, считаю что оставшиеся игроки полностью внесли вклад в результат.

In [19]:
UNIQUE_PLAYERS_2019 = np.unique(np.asarray(np.concatenate(df_results[df_results['dateStart'].dt.year == 2019]['playersId'].values), dtype=np.int64))
UNIQUE_PLAYERS_2020 = np.unique(np.asarray(np.concatenate(df_results[df_results['dateStart'].dt.year == 2020]['playersId'].values), dtype=np.int64))
INTERSECTED_PLAYERS = np.intersect1d(UNIQUE_PLAYERS_2019, UNIQUE_PLAYERS_2020)

NUM_UNIQUE_PLAYERS_2019 = len(UNIQUE_PLAYERS_2019)
NUM_UNIQUE_PLAYERS_2020 = len(UNIQUE_PLAYERS_2020)

PLAYER_ID_TO_FEATURE_ID_2019 = {player: i for i, player in enumerate(UNIQUE_PLAYERS_2019)}
PLAYER_ID_TO_FEATURE_ID_2020 = {player: i for i, player in enumerate(UNIQUE_PLAYERS_2020)}

print(
    f'Число уникальных игроков в 2019 – {len(UNIQUE_PLAYERS_2019)}',
    f'Число уникальных игроков в 2020 – {len(UNIQUE_PLAYERS_2020)}',
    f'Пересечение – {len(INTERSECTED_PLAYERS)}', sep='\n'
)

Число уникальных игроков в 2019 – 59271
Число уникальных игроков в 2020 – 28301
Пересечение – 24089


In [20]:
df_results['playersId'] = df_results['playersId'].apply(lambda x: np.intersect1d(x, INTERSECTED_PLAYERS))

In [21]:
df_results = df_results[df_results['playersId'].map(len) > 0]

Посчитаю итоговое число вопросов и наблюдений которое будет в датасете. Мой набор данных в итоге будет представлять из себя матрицу где каждое наблюдение отвечает за результат игрока в турнире, а в качестве признаков будут идти сначала индикаторы игроков, далее индикаторы вопросов. Например для набора данных где присутствует 3 игрока и 3 вопроса, наблюдение 010001 будет означать что игрок №2 ответил на вопрос №3,таргетом будет также бинарное значение 0/1 характеризующее правильность ответа. 

В ходе разметки данных было замечено что некоторые игры для разных команд имеют разное число вопрсово. Насколько я понял это перестрелки, то есть задавались доп вопросы исходя из которых и получался итоговый балл. Соотвественно буду считать что в игре было максиммальное среди всех команд число вопрсов, то есть если кто-то даже не участвоввал в перестрелке, то он просто не ответил на этот вопрос.

In [22]:
QUESTIONS_NUM_2019 = 0
QUESTIONS_NUM_2020 = 0

QUESTION_ID_TO_FEATURE_ID_2019 = {}
QUESTION_ID_TO_FEATURE_ID_2020 = {}
j = 0
l = 0
for i, group in df_results.groupby(['id']):
    group['mask_max_len'] = group['mask'].map(len).max()
    group['mask_len'] = group['mask'].map(len)
    group_first = group[group['mask_max_len'] == group['mask_len']].head(1)
    mask_len = len(group_first['mask'].values[0])
    if int(group_first['dateStart'].dt.year) == 2019:
        QUESTIONS_NUM_2019 += mask_len
        QUESTION_ID_TO_FEATURE_ID_2019[i] = [k + j for k in range(mask_len)]
        j += mask_len
        
    else:
        QUESTIONS_NUM_2020 += mask_len
        QUESTION_ID_TO_FEATURE_ID_2020[i] = [k + l for k in range(mask_len)]
        l += mask_len
    
# Число вопрсоов
df_results['mask_len'] = df_results['mask'].map(len)

# Число игроков в команде
df_results['players_len'] = df_results['playersId'].map(len)

# Число наблюдений на игру
df_results['obs_per_game'] = df_results['mask_len'] * df_results['players_len']

OBS_NUM_2019 = df_results[df_results.dateStart.dt.year == 2019]['obs_per_game'].sum()
OBS_NUM_2020 = df_results[df_results.dateStart.dt.year == 2020]['obs_per_game'].sum()


In [23]:
print(f'Число вопросов в 2019 году {QUESTIONS_NUM_2019} \nЧисло вопросов в 2020 году {QUESTIONS_NUM_2020}')
print(f'Число наблюдений для 2019 года {OBS_NUM_2019}\nЧисло наблюдений для 2020 года {OBS_NUM_2020}')

Число вопросов в 2019 году 33427 
Число вопросов в 2020 году 7767
Число наблюдений для 2019 года 17083344
Число наблюдений для 2020 года 4157521


Разметка таблицы объекты–признаки

In [24]:
df_results.head(2)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,...,dateEnd,type,season,orgcommittee,synchData,questionQty,playersId,mask_len,players_len,obs_per_game
280058,"{'id': 45556, 'name': 'Рабочее название', 'tow...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","{'name': 'Рабочее название', 'town': {'id': 28...",28.0,"{'id': 56392, 'venue': {'id': 3030, 'name': 'С...",1.0,"[{'id': 91169, 'questionNumber': 15, 'answer':...",[],"[{'flag': 'Б', 'usedRating': 13507, 'rating': ...",4772,...,2019-01-09 16:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[6212, 15456, 18036, 18332, 22799, 26089]",36,6,216
280059,"{'id': 1030, 'name': 'Сборная Бутана', 'town':...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, ...","{'name': 'Сборная Бутана', 'town': {'id': 346,...",25.0,"{'id': 56690, 'venue': {'id': 3151, 'name': 'У...",5.5,[],[],"[{'flag': None, 'usedRating': 13058, 'rating':...",4772,...,2019-01-09 16:00:00+00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[1584, 1585, 10998, 16206, 40840]",36,5,180


# 2 Baseline модель

In [25]:
from collections import deque
def extract_features(data, question_id_map):
    res_player_id = deque([])
    res_question_id = deque([])
    res_target_id = deque([])
    res_index = deque([])
    for i, tup in tqdm(data.iterrows()):
        season = tup['season']
        game_id = tup['id']
        
        question_id = question_id_map[game_id]
        
        # расширение маски до максимальной среди игр данного id
        mask = list(tup['mask']) 
        mask = mask + [0] * (len(question_id) - len(mask))
        
        # упаковка маски и id вопросов
        target = list(zip(mask, question_id))
        
        # id игроков
        players = list(tup['playersId'])
        
        obs = list(product(players, target))
        
        players_obs = [p[0] for p in obs]
        question_obs = [q[1][1] for q in obs]
        target = [t[1][0] for t in obs]
        
        # game team index
        index = [i] * len(target)
        
        res_player_id.extend(players_obs)
        res_question_id.extend(question_obs)
        res_target_id.extend(target)
        res_index.extend(index)
    return pd.DataFrame({
        'player_id': res_player_id,
        'question_id': res_question_id,
        'target': res_target_id,
        'index': res_index}, dtype=np.int32)

In [26]:
df_2019 = extract_features(df_results[df_results.dateStart.dt.year == 2019], QUESTION_ID_TO_FEATURE_ID_2019)

79692it [00:14, 5430.18it/s]


In [27]:
# Используем OneHot - вектора по ID игроков и вопросов в качестве признаков для обучения модели
ohe_player = OneHotEncoder()
ohe_player_encoded = ohe_player.fit_transform(df_2019.player_id.to_numpy().reshape(-1, 1))

ohe_questions = OneHotEncoder()
ohe_questions_encoded = ohe_questions.fit_transform(df_2019.question_id.to_numpy().reshape(-1, 1))

X_train = sparse.csr_matrix(hstack([ohe_player_encoded, ohe_questions_encoded]))
y_train = df_2019[['target']]

In [28]:
X_train

<20410413x57516 sparse matrix of type '<class 'numpy.float64'>'
	with 40820826 stored elements in Compressed Sparse Row format>

In [29]:
X_train.shape, y_train.shape

((20410413, 57516), (20410413, 1))

In [30]:
model_lr = LogisticRegression(n_jobs=-1, max_iter=150, verbose=2)
model_lr.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.1min finished


LogisticRegression(max_iter=150, n_jobs=-1, verbose=2)

In [31]:
def get_players_rank_df(coef, ohe_player, df_players):
    players_rank = pd.DataFrame(
    {'player_id': ohe_player.categories_[0],
     'score': coef[: ohe_player_encoded.shape[1]]
    })
    df_players['fio'] = df_players.apply(lambda x: str(x[3])+' '+ str(x[1])+' '+ str(x[2]), axis=1)
    map_players_id = df_players[['id', 'fio']].to_dict()['fio']
    players_rank['name'] = players_rank['player_id'].map(map_players_id)
    return players_rank

In [32]:
players_rank = get_players_rank_df(-model_lr.coef_[0], ohe_player, df_players)
players_rank.head()

,player_id,score,name
0,15,-0.158260,Абарников Олег Игоревич
1,16,-1.133803,Абасалиев Азер Абасали оглы
2,23,-0.732028,Абащенко Андрей Николаевич
3,31,-0.799772,Абасова Ситара Фахраддин гызы
4,35,-0.715112,Абгарян Нарек Гагикович


In [33]:
players_rank.sort_values('score', ascending=False).head(30)

,player_id,score,name
19188,207704,4.135565,Ильина Анастасия Михайловна
19191,207707,4.135565,Сычкин Антон Денисович
19186,207702,4.135565,Безъязыкова Дарья Александровна
19187,207703,4.135565,Петрушова Дарья Александровна
19190,207706,4.135565,Насыров Иван Владимирович
19189,207705,4.135565,Малыгина Кристина Александровна
18754,207015,4.057801,Иванов Андрей Антонович
18755,207018,4.057801,Карлова Эвелина Алексеевна
18756,207019,4.057801,Крук Марк Владимирович
19197,207713,4.028760,Биктимирова Екатерина Олеговна


In [34]:
players_rank = players_rank.set_index('player_id')

# 3 Оценка качества

Для предсказания результатов нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд будем использовать рейтинг команд, рассчитанный как среднее по рейтингам их членов.

В качестве метрики качества на тестовом наборе будем считать ранговые корреляции Спирмена и Кендалла между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров за 2020 год.

Турниры где участвовало меньше двух команд я буду пропускать.

In [35]:
def correlation(data, players_rank):
    """
    data: df_results сборный датасет где каждое наблюдение это игра команды
    players_rank: ранг игрока player_id:index, score, name
    """
    spearman_values = deque([])
    kendalltau_values = deque([])

    for i, group in tqdm(data.groupby('id')):
        if group.shape[0] < 2:
            continue

        model_positions = []
        exact_positions = []
        for j, team in group.iterrows():
            team_position = team['position']
            team_players = team['playersId']

            model_position = []
            for player in team_players:
                model_position.append(players_rank.loc[player]['score'])
            model_position = np.mean(model_position)

            exact_positions.append(team_position)
            model_positions.append(model_position)

        spearman_values.append(spearmanr(model_positions, exact_positions)[0])
        kendalltau_values.append(kendalltau(model_positions, exact_positions)[0])
    return(spearman_values, kendalltau_values)

In [203]:
spearman_values, kendalltau_values = correlation(df_results[df_results.dateStart.dt.year == 2020], players_rank)
print(f'Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: {np.mean(spearman_values)}')
print(f'Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: {np.mean(kendalltau_values)}')

100%|██████████| 171/171 [00:13<00:00, 12.45it/s]

Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: 0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: 0.5915188730769857


# 4 EM Алгоритм

## E - шаг

На данном шаге происходит обновление таргетов по следующему правилу:

В случае если команда ответила на вопрос:
$$z_{ij} = \frac{\sigma(s_i + q_j)}{1 - \prod_{i'\in t}(1 - \sigma(s_{i'} + q_j))}$$


Tсли команда НЕ ответила на вопрос:
$$z_{ij} = 0$$

Исходно эти параметры берутся из baseline модели, в регрессии это веса.
<br>$s_i$ – скилл игрока
<br>$q_j$ – сложность вопроса
<br>$z_{ij}$def sigmoid(x):
    return 1 / (1 + np.exp(-x)) – ответ игрока i на вопрос j c учетом принадлежности к команде
<br>Обучающая матрица не меняется. Меняются только веса $s_i$ и $q_j$
</tab><br>Числитель это вероятность i игроком ответить на вопрос j
<br>Знаменатель – вероятность того, что остальные игроки не ответили на вопросы. Рассмотрим пограничные случаи: <pre><br>1) Пусть все игроки команды ответили на вопрос с высокой вероятностью (близкой к 1), тогда игрок в числителе никак не повлиял на решение данной задачи. Знаменатель превращается в 1.  <br>2) Ни один из игроков кроме того, который в числителе не дал ответа на вопрос. Значит в знаменателе получается число близкое к 0. Следовательно игрок в числителе внес огромный вклад в решение задачи.

In [36]:
ohe_dict_player = {player: i for i, player in enumerate(ohe_player.categories_[0])}
ohe_dict_question = {question: i + len(ohe_dict_player) for i, question in enumerate(ohe_questions.categories_[0])}

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_product(s_i, q):
    """
    s_i: s_i' силы всех игроков кроме текущего 
    q_j: q_j' сложность текущего вопроса
    """
    s_i = np.asarray(s_i) + q
    return np.product(1 - np.asarray(list(map(sigmoid, s_i))))

def update_target(params, train_data, data_prepare):
    """
    params: параметры модели логрег
    n_users: число игроков
    n_questions: число вопрсов
    train_data: тренировочный набор данных
    data_prepare: набор данных из которого был создан тренировочный
    """
    target = deque([])
    for i, group in tqdm(train_data.groupby('index')):
        players = data_prepare.loc[i]['playersId']
        for sub_group in group.values:
            current_player = sub_group[0]
            cuttent_question = sub_group[1]
            other_players = np.setdiff1d(players, current_player)
            
            
            s_i = params[ohe_dict_player[current_player]]
            q_j = params[ohe_dict_question[cuttent_question]]
            
            s_i_others = list(map(lambda x: ohe_dict_player[x], other_players))
            numerator = sigmoid(s_i + q_j)
            denominator = 1 - sigmoid_product(s_i_others, q_j)
            target.append(numerator / denominator)
    return target

In [159]:
updated_target = update_target(model_lr.coef_[0], df_2019, df_results)

100%|██████████| 79692/79692 [26:34<00:00, 49.99it/s]  


## M - шаг

На данном шаге происходит обучение линейной регрессии на новый таргет

## Обучение по EM схеме logreg

In [450]:
def optimal_threshold(true_target, predictions):
    fpr, tpr, thresholds = roc_curve(true_target, predictions)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

In [451]:
model_em = LogisticRegression(n_jobs=-1, max_iter=50, verbose=2, warm_start=True)
model_em.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.4min finished


LogisticRegression(max_iter=50, n_jobs=-1, verbose=2, warm_start=True)

In [452]:
players_rank = get_players_rank_df(-model_em.coef_[0], ohe_player, df_players)
players_rank = players_rank.set_index('player_id')
players_rank.head()

spearman_values, kendalltau_values = correlation(df_results[df_results.dateStart.dt.year == 2020], players_rank)
print(f'Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: {np.mean(spearman_values)}')
print(f'Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: {np.mean(kendalltau_values)}')

100%|██████████| 171/171 [00:13<00:00, 12.39it/s]

Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: 0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: 0.5915188730769857


In [453]:
params = [model_em.coef_[0]]
correlations = {'spearman': [np.mean(spearman_values)], 'kendalltau': [np.mean(kendalltau_values)]}

updated_target = update_target(model_em.coef_[0], df_2019, df_results)
y_train = np.asarray(updated_target)
y_train[y_train > 1] = 1

treshold = optimal_threshold(df_2019['target'], y_train)
y_train = (y_train > treshold).astype(np.int8)

100%|██████████| 79692/79692 [26:00<00:00, 51.06it/s]  


In [ ]:
for i in tqdm(range(3)):
    model_em.fit(X_train, y_train)
    params.append(model_em.coef_[0])
    
    players_rank = get_players_rank_df(-model_em.coef_[0], ohe_player, df_players)
    players_rank = players_rank.set_index('player_id')
    spearman_values, kendalltau_values = correlation(df_results[df_results.dateStart.dt.year == 2020], players_rank)
    
    correlations['spearman'].append(np.mean(spearman_values))
    correlations['kendalltau'].append(np.mean(kendalltau_values))
    
    if i < 2:
        updated_target = update_target(model_em.coef_[0], df_2019, df_results)
        y_train = np.asarray(updated_target)
        y_train[y_train > 1] = 1
        treshold = optimal_threshold(df_2019['target'], y_train)
        y_train = (y_train > treshold).astype(np.int8)

In [456]:
params

[array([-1.47413472e+00,  3.65575317e-01, -4.91783519e-03, ...,
        -1.56596353e+01, -1.99456252e+01, -1.99456252e+01]),
 array([ -1.5782508 ,   0.46520859,  -0.19921165, ...,  -9.65012146,
        -13.69896342, -13.69896342]),
 array([-0.53088113,  0.36580492, -0.41320861, ..., -7.13378687,
        -6.68435088, -6.68435088]),
 array([ 0.43073791,  0.26054881, -0.59063446, ..., -8.21204736,
        -8.29768185, -8.29768185])]

In [475]:
for i in range(len(correlations['spearman'])):

    print(f'\nНомер итерации = {i}')
    print(f"Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:{correlations['spearman'][i]}")
    print(f"Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:{correlations['kendalltau'][i]}")


Номер итерации = 0
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.5915188730769857

Номер итерации = 1
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.5915188730769857

Номер итерации = 2
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.5915188730769857

Номер итерации = 3
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.7489875741499045
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.5915188730769857


Обучение не дало прироста качества

## Обучение по EM схеме linreg

За основу возьмем результат работы baseline logreg

In [48]:
model_linreg = SGDRegressor(warm_start=True, loss='squared_loss', max_iter=50, verbose=2)

In [39]:
updated_target = update_target(model_lr.coef_[0], df_2019, df_results)
y_train = np.asarray(updated_target)
y_train[y_train > 1] = 1

100%|██████████| 79692/79692 [25:35<00:00, 51.89it/s]  


In [51]:
model_linreg.fit(X_train, y_train)

params_linreg = [model_linreg.coef_]

players_rank = get_players_rank_df(-model_linreg.coef_, ohe_player, df_players)
players_rank = players_rank.set_index('player_id')
players_rank.head()


spearman_values, kendalltau_values = correlation(df_results[df_results.dateStart.dt.year == 2020], players_rank)
correlations_linreg = {'spearman': [np.mean(spearman_values)], 'kendalltau': [np.mean(kendalltau_values)]}

print(f'Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: {np.mean(spearman_values)}')
print(f'Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: {np.mean(kendalltau_values)}')

-- Epoch 1
Norm: 7.39, NNZs: 57516, Bias: 0.588434, T: 20410413, Avg. loss: 0.032671
Total training time: 6.73 seconds.
-- Epoch 2
Norm: 9.50, NNZs: 57516, Bias: 0.590529, T: 40820826, Avg. loss: 0.024743
Total training time: 13.54 seconds.
-- Epoch 3
Norm: 10.59, NNZs: 57516, Bias: 0.591290, T: 61231239, Avg. loss: 0.022258
Total training time: 21.06 seconds.
-- Epoch 4
Norm: 11.24, NNZs: 57516, Bias: 0.592085, T: 81641652, Avg. loss: 0.020994
Total training time: 28.10 seconds.
-- Epoch 5
Norm: 11.65, NNZs: 57516, Bias: 0.592936, T: 102052065, Avg. loss: 0.020253
Total training time: 34.73 seconds.
-- Epoch 6
Norm: 11.92, NNZs: 57516, Bias: 0.593056, T: 122462478, Avg. loss: 0.019783
Total training time: 41.30 seconds.
-- Epoch 7
Norm: 12.11, NNZs: 57516, Bias: 0.593157, T: 142872891, Avg. loss: 0.019469
Total training time: 47.75 seconds.
-- Epoch 8
Norm: 12.25, NNZs: 57516, Bias: 0.593204, T: 163283304, Avg. loss: 0.019252
Total training time: 54.42 seconds.
-- Epoch 9
Norm: 12.35,

100%|██████████| 171/171 [00:13<00:00, 12.28it/s]

Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: 0.6385683501634088
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: 0.4858989381603166


In [52]:
for i in range(3):
    model_linreg.fit(X_train, y_train)
    params_linreg.append(model_linreg.coef_)
    
    players_rank = get_players_rank_df(-model_linreg.coef_, ohe_player, df_players)
    players_rank = players_rank.set_index('player_id')
    spearman_values, kendalltau_values = correlation(df_results[df_results.dateStart.dt.year == 2020], players_rank)
    
    correlations_linreg['spearman'].append(np.mean(spearman_values))
    correlations_linreg['kendalltau'].append(np.mean(kendalltau_values))
    
    if i < 2:
        updated_target = update_target(model_linreg.coef_, df_2019, df_results)
        y_train = np.asarray(updated_target)
        y_train[y_train > 1] = 1


-- Epoch 1
Norm: 12.49, NNZs: 57516, Bias: 0.593452, T: 20410413, Avg. loss: 0.018918
Total training time: 6.61 seconds.
-- Epoch 2
Norm: 12.55, NNZs: 57516, Bias: 0.593632, T: 40820826, Avg. loss: 0.018800
Total training time: 13.14 seconds.
-- Epoch 3
Norm: 12.59, NNZs: 57516, Bias: 0.593338, T: 61231239, Avg. loss: 0.018741
Total training time: 19.68 seconds.
-- Epoch 4
Norm: 12.61, NNZs: 57516, Bias: 0.593558, T: 81641652, Avg. loss: 0.018701
Total training time: 26.21 seconds.
-- Epoch 5
Norm: 12.63, NNZs: 57516, Bias: 0.593884, T: 102052065, Avg. loss: 0.018676
Total training time: 32.98 seconds.
-- Epoch 6
Norm: 12.64, NNZs: 57516, Bias: 0.593924, T: 122462478, Avg. loss: 0.018657
Total training time: 40.54 seconds.
Convergence after 6 epochs took 40.54 seconds


100%|██████████| 79692/79692 [28:26<00:00, 46.69it/s]   


-- Epoch 1
Norm: 6.56, NNZs: 57516, Bias: 0.504600, T: 20410413, Avg. loss: 0.002635
Total training time: 6.97 seconds.
-- Epoch 2
Norm: 4.64, NNZs: 57516, Bias: 0.504816, T: 40820826, Avg. loss: 0.001186
Total training time: 13.64 seconds.
-- Epoch 3
Norm: 3.63, NNZs: 57516, Bias: 0.505094, T: 61231239, Avg. loss: 0.001018
Total training time: 20.32 seconds.
-- Epoch 4
Norm: 3.04, NNZs: 57516, Bias: 0.505319, T: 81641652, Avg. loss: 0.001000
Total training time: 26.98 seconds.
-- Epoch 5
Norm: 2.67, NNZs: 57516, Bias: 0.505471, T: 102052065, Avg. loss: 0.001012
Total training time: 33.63 seconds.
-- Epoch 6
Norm: 2.43, NNZs: 57516, Bias: 0.505655, T: 122462478, Avg. loss: 0.001026
Total training time: 40.57 seconds.
-- Epoch 7
Norm: 2.28, NNZs: 57516, Bias: 0.505741, T: 142872891, Avg. loss: 0.001039
Total training time: 47.54 seconds.
Convergence after 7 epochs took 47.54 seconds


100%|██████████| 79692/79692 [25:59<00:00, 51.09it/s]  


-- Epoch 1
Norm: 1.48, NNZs: 57516, Bias: 0.505050, T: 20410413, Avg. loss: 0.001016
Total training time: 7.06 seconds.
-- Epoch 2
Norm: 1.30, NNZs: 57516, Bias: 0.505127, T: 40820826, Avg. loss: 0.000977
Total training time: 13.52 seconds.
-- Epoch 3
Norm: 1.22, NNZs: 57516, Bias: 0.505196, T: 61231239, Avg. loss: 0.000972
Total training time: 20.34 seconds.
-- Epoch 4
Norm: 1.18, NNZs: 57516, Bias: 0.505239, T: 81641652, Avg. loss: 0.000971
Total training time: 27.34 seconds.
-- Epoch 5
Norm: 1.16, NNZs: 57516, Bias: 0.505282, T: 102052065, Avg. loss: 0.000972
Total training time: 34.07 seconds.
-- Epoch 6
Norm: 1.15, NNZs: 57516, Bias: 0.505217, T: 122462478, Avg. loss: 0.000972
Total training time: 40.58 seconds.
Convergence after 6 epochs took 40.58 seconds


100%|██████████| 171/171 [00:13<00:00, 12.96it/s]


In [55]:
for i in range(len(correlations_linreg['spearman'])):

    print(f'\nНомер итерации = {i}')
    print(f"Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:{correlations_linreg['spearman'][i]}")
    print(f"Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:{correlations_linreg['kendalltau'][i]}")


Номер итерации = 0
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.6385683501634088
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.4858989381603166

Номер итерации = 1
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.6408380687145345
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.4882535636374567

Номер итерации = 2
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.5620497336091993
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.42434255334822524

Номер итерации = 3
Средний коэффициент ранговой корреляции Спирмена на тестовом множестве:0.1751191718053372
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве:0.12336636512243908


Все стало только хуже... 

# 5 Топ самых сложных и легких турниров по вопросам 

In [108]:
df_2019['question_weight_id'] = df_2019['question_id'].map(ohe_dict_question)
df_tournaments['weight'] = None

In [122]:
for i, group in tqdm(df_2019.groupby('index')):
    question_id = group['question_weight_id'].unique()
    questions_weight = np.mean(model_lr.coef_[0][question_id])
    df_tournaments.loc[df_results.loc[i]['id'], 'weight'] = questions_weight

100%|██████████| 79692/79692 [00:46<00:00, 1711.33it/s]


In [127]:
tmp_tournaments = df_tournaments[~df_tournaments['weight'].isna()][['name', 'weight']]

In [130]:
tmp_tournaments.sort_values('weight')[:20]

,name,weight
6149,Чемпионат Санкт-Петербурга. Первая лига,-4.489098
5554,Гран-при Славянки. Общий зачёт,-2.762239
5553,Славянка без раздаток. Общий зачёт,-2.494519
5864,Гран-при Славянки. Общий зачет,-2.22424
6085,Серия Гран-при. Общий зачёт,-2.202348
5928,Угрюмый Ёрш,-2.128013
5684,Синхрон высшей лиги Москвы,-2.087865
5159,Первенство правого полушария,-1.895169
6101,Воображаемый музей,-1.892226
5587,Записки охотника,-1.624599


In [131]:
tmp_tournaments.sort_values('weight', ascending=False)[:20]

,name,weight
5013,(а)Синхрон-lite. Лига старта. Эпизод V,2.34966
5702,(а)Синхрон-lite. Лига старта. Эпизод IX,2.328098
5698,(а)Синхрон-lite. Лига старта. Эпизод VII,2.310327
5706,(а)Синхрон-lite. Лига старта. Эпизод XI,2.290972
5009,(а)Синхрон-lite. Лига старта. Эпизод III,2.24123
5438,Синхрон Лиги Разума,2.063039
5011,(а)Синхрон-lite. Лига старта. Эпизод IV,2.039911
5704,(а)Синхрон-lite. Лига старта. Эпизод X,1.981407
6003,Второй тематический турнир имени Джоуи Триббиани,1.923832
5012,Школьный Синхрон-lite. Выпуск 2.5,1.913714


В целом сложность вопрсов соотвествует логике, наиболее сложные чемпионаты шли с отрицательными весами, по ним отвечали хуже. Сложным турнирам соотвествуют "Чемпионат Санкт-Петербурга. Первая лига", "Чемпионат Мира. Этап 2 Группа С" и тд

Самые легкие "всплыли" наверх, им соотвествуют такие турниры как "Школьная лига", "Лига Вузов"